# Integration pipeline of clinical data only

## Parameter definition

In [1]:
results_path = "path"
permute = "_label_permutation" # "", "_label_permutation", "_feature_permutation";  "" if you do not want permutation

NUMBER_REPETITIONS = 200
NUMBER_TEST_SAMPLES = 5 # per class
NUMBER_TRAIN_SAMPLES = 30 # per class per fold execution
NUMBER_INTERNAL_FOLDS = 5
var_threshold = 0.0 # 0.0 to remove only constant features
classifier = "SVM" # "SVM", "NN", "RF"

## Data loading and minimal preprocessing

In [2]:
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from scipy.io import savemat

random_state = 1
random.seed(random_state)
rng = np.random.RandomState(random_state)

# load age and gender of the samples
sampleData = pd.read_csv("path\sampleData.csv")
ages = sampleData["Age"].to_numpy().reshape(-1, 1)
print("Age, NaNs: ", sum(np.isnan(ages)))
genders = OneHotEncoder(sparse=False).fit_transform(sampleData["Sex"].to_numpy().reshape(-1, 1))
print("Gender, NaNs: ", sum(np.isnan(genders)))

# load targets
targets = sampleData["Status"] == "Tumor" # ids are in the same order
targets = LabelEncoder().fit_transform(targets)

# concatenate all data
data = np.concatenate((ages, genders), axis=1)
uni_data, _, unique_inverse = np.unique(data, return_index=True, return_inverse=True, axis=0)
print((uni_data[unique_inverse] != data).any()) # because of the presence of NaNs, we will take care of them later
print(data.shape)     

Age, NaNs:  [3]
Gender, NaNs:  [0 0]
True
(151, 3)


## Integration analysis and permutation analysis

In [3]:
from sklearn.utils import shuffle
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, matthews_corrcoef

tumor_samples = data[targets==1,:]
healthy_samples = data[targets==0,:]
cv = StratifiedKFold(n_splits=NUMBER_INTERNAL_FOLDS, shuffle=True, random_state=random_state)

if classifier == "SVM":
    mdl = SVC(kernel="linear", random_state=rng)
    params = {
        "model__C": np.logspace(-10, 10, num=10)
    }
elif classifier == "NN":
    mdl = MLPClassifier(solver="lbfgs", random_state=rng)
    params = {
        "model__alpha": 10.0 ** -np.arange(1, 7)
    }
else:
    mdl = RandomForestClassifier(random_state=rng)
    params = {
        "model__n_estimators": [10, 100, 500, 1000],
        "model__max_depth": [3, 5, 8, None]
    }

pipe = Pipeline(steps=[
    ("oversampling", RandomOverSampler(sampling_strategy="minority", random_state=random_state)), 
    ("undersampling", RandomUnderSampler(sampling_strategy={0:NUMBER_TRAIN_SAMPLES, 1:NUMBER_TRAIN_SAMPLES}, 
                                         random_state=random_state)),
    ("separate_pipelines", ColumnTransformer([
        ("ages", KNNImputer(n_neighbors=5, weights="uniform", metric="nan_euclidean"), [0]), # ages
        ("genders", "passthrough", [1, 2])
    ])),
    ("standardise", StandardScaler()), 
    ('model',  mdl)
])


params = {
    "svc__C": lambdas
}

accuracies = np.zeros(NUMBER_REPETITIONS)
MCCs = np.zeros(NUMBER_REPETITIONS)
Ytrue = np.zeros((NUMBER_REPETITIONS, NUMBER_TEST_SAMPLES * 2))
Ypred = np.zeros((NUMBER_REPETITIONS, NUMBER_TEST_SAMPLES * 2))
testSamples = np.zeros((NUMBER_REPETITIONS, NUMBER_TEST_SAMPLES * 2))
betas = list()    

for rep in range(NUMBER_REPETITIONS):
    test_idx1 = random.sample(range(len(healthy_samples)), NUMBER_TEST_SAMPLES)
    test_idx2 = random.sample(range(len(tumor_samples)), NUMBER_TEST_SAMPLES)
    testSamples[rep] = np.concatenate((test_idx1, test_idx2)) + 1 # for use in MATLAB
       
    Xtest = np.concatenate((healthy_samples[test_idx1,:], tumor_samples[test_idx2,:]))
    ytest = [i for i in range(2) for _ in range(NUMBER_TEST_SAMPLES)]
    
    train_healthy = np.delete(healthy_samples, test_idx1, axis=0)
    train_tumor = np.delete(tumor_samples, test_idx2, axis=0)
    Xtrain = np.concatenate((train_healthy, train_tumor))
    Xtrain, ytrain = shuffle(Xtrain, np.concatenate(([0] * len(train_healthy), [1] * len(train_tumor))), 
                             random_state=random_state)
    
    if permute == "_label_permutation":
        ys = np.concatenate((ytrain, ytest))
        random.shuffle(ys)
        ytrain, ytest = ys[:len(ytrain)], ys[-len(ytest):]
    elif permute == "_feature_permutation":
        raise NotImplementedError
    else:
        pass
    
    gs = GridSearchCV(pipe, param_grid=params, cv=cv, refit=True, n_jobs=-1) 
    gs.fit(Xtrain, ytrain)  
    print("Run", rep + 1, gs.best_params_, gs.best_score_) 
    ypred = gs.predict(Xtest)
    accuracies[rep] = accuracy_score(ytest, ypred)
    MCCs[rep] = matthews_corrcoef(ytest, ypred)
    Ytrue[rep] = ytest
    Ypred[rep] = ypred

    if classifier == "SVM":                              
        betas.append({"Beta": gs.best_estimator_["model"].coef_[0]})
    elif classifier == "NN": 
        betas.append({"Beta": np.squeeze(gs.best_estimator_["model"].coefs_[0].dot(gs.best_estimator_["model"].coefs_[1]))})
    else:
        betas.append({"Beta": np.squeeze(np.zeros((Xtrain.shape[0], 1)))}) # too computational expensive to calculate feature
                                                                           # importance
    
print("Average accuracy: {} with standard deviation: {}".format(np.mean(accuracies), np.std(accuracies)))
print("Average MCC: {} with standard deviation: {}".format(np.mean(MCCs), np.std(MCCs)))

Run 1 {'svc__C': 59948425.03189421} 0.5179802955665025
Run 2 {'svc__C': 12.915496650148826} 0.4596059113300493
Run 3 {'svc__C': 1e-10} 0.6187192118226601
Run 4 {'svc__C': 10000000000.0} 0.5958128078817733
Run 5 {'svc__C': 0.07742636826811278} 0.5261083743842365
Run 6 {'svc__C': 359381.36638046405} 0.41847290640394086
Run 7 {'svc__C': 12.915496650148826} 0.49556650246305417
Run 8 {'svc__C': 359381.36638046405} 0.6169950738916257
Run 9 {'svc__C': 59948425.03189421} 0.5598522167487685
Run 10 {'svc__C': 59948425.03189421} 0.7598522167487685
Run 11 {'svc__C': 10000000000.0} 0.5091133004926107
Run 12 {'svc__C': 359381.36638046405} 0.5674876847290641
Run 13 {'svc__C': 1e-10} 0.5243842364532019
Run 14 {'svc__C': 12.915496650148826} 0.4455665024630542
Run 15 {'svc__C': 10000000000.0} 0.6738916256157635
Run 16 {'svc__C': 1e-10} 0.5815270935960591
Run 17 {'svc__C': 59948425.03189421} 0.6241379310344828


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Run 18 {'svc__C': 359381.36638046405} 0.5608374384236454
Run 19 {'svc__C': 12.915496650148826} 0.49679802955665026
Run 20 {'svc__C': 59948425.03189421} 0.6236453201970443
Run 21 {'svc__C': 10000000000.0} 0.6315270935960591
Run 22 {'svc__C': 12.915496650148826} 0.553448275862069


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Run 23 {'svc__C': 1e-10} 0.6086206896551725
Run 24 {'svc__C': 359381.36638046405} 0.632512315270936
Run 25 {'svc__C': 359381.36638046405} 0.7231527093596058
Run 26 {'svc__C': 1e-10} 0.5810344827586207
Run 27 {'svc__C': 0.07742636826811278} 0.5886699507389161


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Run 187 {'svc__C': 12.915496650148826} 0.5886699507389161
Run 188 {'svc__C': 59948425.03189421} 0.6330049261083743
Run 189 {'svc__C': 1e-10} 0.49556650246305417
Run 190 {'svc__C': 12.915496650148826} 0.5679802955665025
Run 191 {'svc__C': 0.07742636826811278} 0.4891625615763546


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Run 192 {'svc__C': 359381.36638046405} 0.5948275862068966
Run 193 {'svc__C': 12.915496650148826} 0.5246305418719212
Run 194 {'svc__C': 359381.36638046405} 0.496551724137931
Run 195 {'svc__C': 12.915496650148826} 0.5467980295566501
Run 196 {'svc__C': 59948425.03189421} 0.6238916256157635
Run 197 {'svc__C': 59948425.03189421} 0.4896551724137931
Run 198 {'svc__C': 59948425.03189421} 0.6298029556650246
Run 199 {'svc__C': 359381.36638046405} 0.5403940886699508
Run 200 {'svc__C': 12.915496650148826} 0.404679802955665
Average accuracy: 0.48200000000000004 with standard deviation: 0.21535087647836496
Average MCC: -0.01598057192093086 with standard deviation: 0.29265075809356844


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


## Result saving

In [4]:
if classifier == "SVM":
    SVM_results = {
        "predicted": Ypred, 
        "true": Ytrue, 
        "mdl": betas, 
        "accuracies": accuracies, 
        "MCCs": MCCs, 
        "testSamples": testSamples
    }

    savemat(results_path + r"\Int2SVMl2" + "_results_" + "clinical_only" + permute + ".mat", 
            {"SVM_results": SVM_results}, oned_as="column")
elif classifier == "NN":
    NN_results = {
        "predicted": Ypred, 
        "true": Ytrue, 
        "mdl": betas, 
        "accuracies": accuracies, 
        "MCCs": MCCs, 
        "testSamples": testSamples
    }

    savemat(results_path + r"\Int2NN" + "_results_" + "clinical_only" + permute + ".mat", 
            {"NN_results": NN_results}, oned_as="column")
else:
    RF_results = {
        "predicted": Ypred, 
        "true": Ytrue, 
        "mdl": betas, 
        "accuracies": accuracies, 
        "MCCs": MCCs, 
        "testSamples": testSamples
    }

    savemat(results_path + r"\Int2RF" + "_results_" + "clinical_only" + permute + ".mat", 
            {"RF_results": RF_results}, oned_as="column")